### Init Context

In [1]:
from thetaray.api.context import init_context
import datetime
import yaml

import logging
logging.basicConfig(level=logging.DEBUG, format='%(message)s')

with open('/thetaray/git/solutions/domains/demo_pay_proc/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']
context = init_context(execution_date=datetime.datetime(1970, 1, 1),
                       spark_conf=spark_config,
                       spark_master='local[*]',)

/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-08-11 18:36:36,604:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-08-11 18:36:36,613:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_human_trafficking_insights, but data encryption is disabled in deployment
2025-08-11 18:36:36,628:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_remittance_customer_insights, but data encryption is disabled in deployment
2025-08-11 18:36:36,638:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but da

### Params

In [2]:
from thetaray.api.dataset import dataset_functions
from thetaray.api.evaluation import evaluate, write_evaluated_activities, publish_evaluated_activities, load_evaluated_activities
from thetaray.common.data_environment import DataEnvironment

from domains.demo_pay_proc.datasets.customer_monthly import customer_monthly_dataset as dataset
from domains.demo_pay_proc.evaluation_flows.ef import evaluation_flow as ef

### Detection

In [3]:
data = dataset_functions.read(context, dataset().identifier, data_environment=DataEnvironment.PUBLIC).drop('tr_timestamp')
evaluated_data = evaluate(context, ef().identifier, data)
evaluated_data = evaluated_data.cache()
evaluated_data.count()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 18:36:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
Hive Session ID = b25be456-3a2f-4082-97a1-de5833f1baf3
25/08/11 18:36:43 INFO SessionState: Hive Session ID = b25be456-3a2f-4082-97a1-de5833f1baf3
2025-08-11 18:36:46,557:INFO:thetaray.common.logging:Applying backwards compatibility patches
2025-08-11 18:36:46.867142: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-11 18:36:46.868944: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-11 18:36:46.872085: I external/local

/thetaray/venv/lib64/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.0.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/thetaray/venv/lib64/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.0.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-08-11 18:36:49,562:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-11 18:36:49,562:INFO:thetaray.common.logging:Rolled back backwards compatibility patches
2025-

/thetaray/venv/lib64/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.0.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
2025-08-11 18:36:49,901:INFO:thetaray.common.logging:Rolling back backwards compatibility patches
2025-08-11 18:36:49,902:INFO:thetaray.common.logging:Rolled back backwards compatibility patches
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60

990

In [4]:
anomalies_count = evaluated_data.filter("algo_score > 0.5").count()
alerts_count = evaluated_data.filter("(algo_score > 0.5) AND (customer_id == 'castaneda_group')").count()
logging.info(f"Number of anomalies: {anomalies_count}")
logging.info(f"Number of alerts: {alerts_count}")

2025-08-11 18:36:59,429:INFO:root:Number of anomalies: 494
2025-08-11 18:36:59,430:INFO:root:Number of alerts: 6


In [5]:
evaluated_data.filter("(algo_score > 0.5) AND (customer_id == 'castaneda_group')").toPandas()

,txn_count,reversals,pipe_account_behaviour_score,year_month_str,avg_txn_amt_ratio,terminals,new_terminal_ratio,rare_currency_pct,domestic_ratio_change,avg_amount,...,algo_reversal_ratio_rank,algo_pipe_account_behaviour_score_rating,algo_pipe_account_behaviour_score_rank,algo_customer_id_rating,algo_customer_id_rank,algo_domestic_ratio_change_rating,algo_domestic_ratio_change_rank,algo_pattern,algo_score,models_metadata
0,10500,42,0.1778,2025-08,0.0007,70,4.7534,2.1163,0.7951,109.924660,...,3,185,6,NaN,NaN,213,5,rare_currency_pct#txn_count_ratio,1.000000,"{""feature_extraction_model"": {""name"": ""manager..."
1,2400,48,0.1955,2025-06,0.0000,50,11.5000,0.0000,0.8004,109.054525,...,1,192,4,NaN,NaN,31,5,reversal_ratio#new_terminal_ratio,0.999999,"{""feature_extraction_model"": {""name"": ""manager..."
2,2063,8,0.1130,2024-12,0.0025,2,0.0000,0.0000,0.7964,110.044881,...,4,5699,1,NaN,NaN,1623,3,pipe_account_behaviour_score#avg_txn_amt_ratio,0.524722,"{""feature_extraction_model"": {""name"": ""manager..."
3,3000,30,0.1339,2025-03,0.0018,15,6.5000,2.8298,0.8020,109.857620,...,2,67,6,NaN,NaN,231,4,rare_currency_pct#reversal_ratio,1.000000,"{""feature_extraction_model"": {""name"": ""manager..."
4,2409,9,0.1788,2025-01,0.0000,2,0.0000,0.0149,0.7891,108.624770,...,1,81,5,NaN,NaN,1790,2,reversal_ratio#domestic_ratio_change,0.909397,"{""feature_extraction_model"": {""name"": ""manager..."
5,2697,8,0.1121,2024-10,0.0000,1,0.0000,0.2381,0.7946,109.394624,...,5,2295,2,NaN,NaN,47,3,rare_currency_pct#pipe_account_behaviour_score,1.000000,"{""feature_extraction_model"": {""name"": ""manager..."


In [6]:
from thetaray.api.evaluation import unblock_evaluation_for_execution_date
unblock_evaluation_for_execution_date(context, ef().identifier, data_environment=DataEnvironment.PUBLIC)

True

In [7]:
write_evaluated_activities(context, evaluated_data, ef().identifier, data_environment=DataEnvironment.PUBLIC)
publish_evaluated_activities(context, ef().identifier, data_environment=DataEnvironment.PUBLIC)

25/08/11 18:40:53 INFO SQLStdHiveAccessController: Created SQLStdHiveAccessController for session context : HiveAuthzSessionContext [sessionString=b25be456-3a2f-4082-97a1-de5833f1baf3, clientType=HIVECLI]
25/08/11 18:40:53 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/demo_pay_proc_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_10
25/08/11 18:40:53 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/demo_pay_proc_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_11
25/08/11 18:40:54 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/demo_pay_proc_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2024_12
25/08/11 18:40:54 ERROR FileUtils: Failed to delete s3a://thetaray-sonar/warehouse/public.db/demo_pay_proc_ef/tr_year=1970/tr_month=1/tr_day=1/tr_date=1970_01_01_00_00_00/tr_partition=2025_01
25/08/11 18:40:54 ERROR Fil

2025-08-11 18:41:01,943:INFO:thetaray.common.logging:_publish_evaluated_activities
           [evaluation_flow_identifier=demo_pay_proc_ef,
            date range [context 1970-01-01 00:00:00, from_job_ts None, to_job_ts None]
            data_environment=DataEnvironment.PUBLIC 
2025-08-11 18:41:03,816:INFO:thetaray.common.logging:finished publishing records for evaluation flow demo_pay_proc_ef
2025-08-11 18:41:04,171:INFO:thetaray.common.logging:histograms_df.count(): 1
2025-08-11 18:41:04,635:INFO:thetaray.common.logging:models_metadata_df.rdd.isEmpty(): False
2025-08-11 18:41:04,864:INFO:thetaray.common.logging:models_metadata_df.count(): 2


True

In [8]:
context.close()